In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("AirlineSpark").master("local[5]").getOrCreate()

25/07/05 06:24:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/05 06:24:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/05 06:24:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
airport_sch = StructType([\
                          StructField("Airport ID", IntegerType(), True),\
                          StructField("Name", StringType(), True),\
                          StructField("City", StringType(), True),\
                          StructField("Country", StringType(), True),\
                          StructField("IATA/FAA", StringType(), True),\
                          StructField("ICAO", StringType(), True),\
                          StructField("Latitude", DecimalType(), True),\
                          StructField("Longitude", DecimalType(), True),\
                          StructField("Alititude", IntegerType(), True),\
                          StructField("Timezone", DecimalType(), True),\
                          StructField("DST", StringType(), True),\
                          StructField("Tz(oslon)", StringType(), True)])

In [9]:
airports = spark.read.csv("certification/airline/airports_mod.dat", sep=",", schema=airport_sch)

In [10]:
airports.show(5)

+----------+--------------------+------------+----------------+--------+----+--------+---------+---------+--------+---+--------------------+
|Airport ID|                Name|        City|         Country|IATA/FAA|ICAO|Latitude|Longitude|Alititude|Timezone|DST|           Tz(oslon)|
+----------+--------------------+------------+----------------+--------+----+--------+---------+---------+--------+---+--------------------+
|         1|              Goroka|      Goroka|Papua New Guinea|     GKA|AYGA|      -6|      145|     5282|      10|  U|Pacific/Port_Moresby|
|         2|              Madang|      Madang|Papua New Guinea|     MAG|AYMD|      -5|      146|       20|      10|  U|Pacific/Port_Moresby|
|         3|         Mount Hagen| Mount Hagen|Papua New Guinea|     HGU|AYMH|      -6|      144|     5388|      10|  U|Pacific/Port_Moresby|
|         4|              Nadzab|      Nadzab|Papua New Guinea|     LAE|AYNZ|      -7|      147|      239|      10|  U|Pacific/Port_Moresby|
|         5|P

In [11]:
airlines_sch = StructType([\
                           StructField("Airline ID", IntegerType(), True),\
                           StructField("Name", StringType(), True),\
                           StructField("Alias", StringType(), True),\
                           StructField("IATA", StringType(), True),\
                           StructField("ICAO", StringType(), True),\
                           StructField("Callsign", StringType(), True),\
                           StructField("Country", StringType(), True),\
                           StructField("Active", StringType(), True)])

In [12]:
airlines = spark.read.csv("certification/airline/Final_airlines", sep=",", schema=airlines_sch)

In [13]:
airlines.show(5)

+----------+--------------------+-----+----+----+--------+--------------+------+
|Airline ID|                Name|Alias|IATA|ICAO|Callsign|       Country|Active|
+----------+--------------------+-----+----+----+--------+--------------+------+
|         1|      Private flight|   \N|   -| N/A|    null|          null|     Y|
|         2|         135 Airways|   \N|null| GNL| GENERAL| United States|     N|
|         3|       1Time Airline|   \N|  1T| RNX| NEXTIME|  South Africa|     Y|
|         4|2 Sqn No 1 Elemen...|   \N|null| WYT|    null|United Kingdom|     N|
|         5|     213 Flight Unit|   \N|null| TFU|    null|        Russia|     N|
+----------+--------------------+-----+----+----+--------+--------------+------+
only showing top 5 rows



In [14]:
routes_sch = StructType([\
                         StructField("Airline", StringType(), True),\
                         StructField("Airline ID", IntegerType(), True),\
                         StructField("Source", StringType(), True),\
                         StructField("Source ID", IntegerType(), True),\
                         StructField("Destination", StringType(), True),\
                         StructField("Destination ID", IntegerType(), True),\
                         StructField("Codeshare", StringType(), True),\
                         StructField("Stops", IntegerType(), True),\
                         StructField("Equipment", StringType(), True)])

In [15]:
routes = spark.read.csv("certification/airline/routes.dat", sep=",", schema=routes_sch)

In [16]:
routes.show(5)

+-------+----------+------+---------+-----------+--------------+---------+-----+---------+
|Airline|Airline ID|Source|Source ID|Destination|Destination ID|Codeshare|Stops|Equipment|
+-------+----------+------+---------+-----------+--------------+---------+-----+---------+
|     2B|       410|   AER|     2965|        KZN|          2990|     null|    0|      CR2|
|     2B|       410|   ASF|     2966|        KZN|          2990|     null|    0|      CR2|
|     2B|       410|   ASF|     2966|        MRV|          2962|     null|    0|      CR2|
|     2B|       410|   CEK|     2968|        KZN|          2990|     null|    0|      CR2|
|     2B|       410|   CEK|     2968|        OVB|          4078|     null|    0|      CR2|
+-------+----------+------+---------+-----------+--------------+---------+-----+---------+
only showing top 5 rows



### A. Find list of Airports operating in the Country India

In [18]:
airports.select(["Airport ID", "Name", "City", "Country"]).where(airports.Country=="India").show()

+----------+--------------------+----------+-------+
|Airport ID|                Name|      City|Country|
+----------+--------------------+----------+-------+
|      2994|           Ahmedabad| Ahmedabad|  India|
|      2995|               Akola|     Akola|  India|
|      2996|          Aurangabad|Aurangabad|  India|
|      2997|Chhatrapati Shiva...|    Mumbai|  India|
|      2998|            Bilaspur|  Bilaspur|  India|
|      2999|                Bhuj|      Bhuj|  India|
|      3000|             Belgaum|   Belgaum|  India|
|      3001|            Vadodara|    Baroda|  India|
|      3002|              Bhopal|    Bhopal|  India|
|      3003|           Bhavnagar| Bhaunagar|  India|
|      3004|               Daman|     Daman|  India|
|      3005|               Deesa|     Deesa|  India|
|      3006|                Guna|      Guna|  India|
|      3007|                 Goa|       Goa|  India|
|      3008|Devi Ahilyabai Ho...|    Indore|  India|
|      3009|            Jabalpur|  Jabalpur|  

### B. Find the list of Airlines having zero stops

In [21]:
routes.select("Airline").where(routes.Stops==0).distinct().show()

+-------+
|Airline|
+-------+
|     3P|
|     DZ|
|     LT|
|     MM|
|     XU|
|     Z9|
|     RG|
|     3K|
|     CI|
|     PC|
|     AZ|
|     FI|
|     JH|
|     PM|
|     SC|
|     6Y|
|     A9|
|     Q7|
|     UA|
|     2Z|
+-------+
only showing top 20 rows



### C. List of Airlines operating with code share

In [25]:
routes.na.drop(subset="Codeshare").show()

+-------+----------+------+---------+-----------+--------------+---------+-----+---------+
|Airline|Airline ID|Source|Source ID|Destination|Destination ID|Codeshare|Stops|Equipment|
+-------+----------+------+---------+-----------+--------------+---------+-----+---------+
|     2P|       897|   GES|     6011|        MNL|          2397|        Y|    0|      320|
|     2P|       897|   MNL|     2397|        GES|          6011|        Y|    0|      320|
|     4M|      3201|   DFW|     3670|        EZE|          3988|        Y|    0|      777|
|     4M|      3201|   EZE|     3988|        DFW|          3670|        Y|    0|      777|
|     4M|      3201|   EZE|     3988|        JFK|          3797|        Y|    0|      777|
|     4M|      3201|   JFK|     3797|        EZE|          3988|        Y|    0|      777|
|     5N|       503|   ARH|     4362|        CSH|          6110|        Y|    0|      AN4|
|     5N|       503|   ARH|     4362|        MMK|          2949|        Y|    0|      AN4|

### D. Which country (or) territory having highest Airports

In [38]:
count_airport = airports.groupBy("Country").agg(count("Country").alias("Count"))

In [41]:
count_airport.orderBy(col("Count").desc()).show(1)

+-------------+-----+
|      Country|Count|
+-------------+-----+
|United States| 1697|
+-------------+-----+
only showing top 1 row



### E. Find the list of Active Airlines in United state

In [43]:
airlines.where(airlines.Country=="United States").show()

+----------+--------------------+-----+----+----+-------------+-------------+------+
|Airline ID|                Name|Alias|IATA|ICAO|     Callsign|      Country|Active|
+----------+--------------------+-----+----+----+-------------+-------------+------+
|         2|         135 Airways|   \N|null| GNL|      GENERAL|United States|     N|
|         9|         3D Aviation|   \N|null| SEC|      SECUREX|United States|     N|
|        10|         40-Mile Air|   \N|  Q5| MLA|     MILE-AIR|United States|     Y|
|        22|      Aloha Airlines|   \N|  AQ| AAH|        ALOHA|United States|     Y|
|        23|   Alaska Island Air|   \N|null| AAK|ALASKA ISLAND|United States|     N|
|        24|   American Airlines|   \N|  AA| AAL|     AMERICAN|United States|     Y|
|        25|Aviation Manageme...|   \N|null| AAM|      AM CORP|United States|     N|
|        26|Atlantis Airlines...|   \N|null| AAO| ATLANTIS AIR|United States|     N|
|        31|Astro Air Interna...|   \N|null| AAV|   ASTRO-PHIL|Un